<a href="https://colab.research.google.com/github/tanmaylaud/biospec/blob/main/PeptideDataProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandarallel

In [ ]:
!pip install pyteomics
from pyteomics import mzml, auxiliary
import pandas as pd
import torch
import glob
import numpy as np
from pandarallel import pandarallel
import re
from scipy.sparse import csr_matrix
from ftplib import FTP
from contextlib import redirect_stdout
import io

     |████████████████████████████████| 204kB 20.6MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/CSE291C00/Stage 1 Analysis 29 tissues Project/'

/content/drive/MyDrive/CSE291C00/Stage 1 Analysis 29 tissues Project


In [ ]:
pandarallel.initialize(progress_bar=False, verbose = 0)

In [ ]:
# finding the minimum value in the m/z array
minval =  float('inf')
maxlen_i = 0
for i, r in enumerate(df['m/z array']):
    if minval > min(r):
        minval = min(r)
        maxlen_i = i
minval

50.443913

In [ ]:
#proteomics dataset used for the project.
!unzip ProteoSAFe-MAESTRO-d5f87289-identified_spectra.zip

Archive:  ProteoSAFe-MAESTRO-d5f87289-identified_spectra.zip
replace MAESTRO-d5f87289-identified_spectra-main.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
df1 = pd.read_csv('MAESTRO-d5f87289-identified_spectra-main.tsv', delimiter='\t')
df1

,cluster index,AllFiles,sum(precursor intensity),RTMean,RTStdErr,parent mass,ScanNumber,ProteosafeFilePath,PrecursorCharge,Original_Path,Mean_KL,Max_KL,Min_SNR2_Peaks,Max_SNR2_Peaks,Spectrum_index,LorikeetFilePath,Peptide,Variant_group,Spectral_network,Num_tags,Protein,FDR
0,5,inputspectra/spec-00018.mgf,650067.0,820.427,0,1074.53,5518,spec-00018.mgf,3,f.MSV000083508/ccms_peak/stomach/Trypsin_HCD_Q...,3.63020,3.63020,41,41,3989,MSV000083508/ccms_peak/stomach/Trypsin_HCD_QEx...,K.IREEYPDR.I,117109,26191.0,102,tr|Q5ST81|Q5ST81_HUMAN,0.000969
1,5,inputspectra/spec-00034.mgf,1259150.0,1021.980,0,1074.53,6917,spec-00034.mgf,3,f.MSV000083508/ccms_peak/stomach/Trypsin_HCD_Q...,3.63020,3.63020,125,125,4979,MSV000083508/ccms_peak/stomach/Trypsin_HCD_QEx...,K.IREEYPDR.I,117109,26191.0,102,tr|Q5ST81|Q5ST81_HUMAN,0.000969
2,5,inputspectra/spec-00046.mgf,460434.0,889.118,0,1074.53,5799,spec-00046.mgf,3,f.MSV000083508/ccms_peak/stomach/Trypsin_HCD_Q...,3.63020,3.63020,44,44,4074,MSV000083508/ccms_peak/stomach/Trypsin_HCD_QEx...,K.IREEYPDR.I,117109,26191.0,102,tr|Q5ST81|Q5ST81_HUMAN,0.000969
3,5,inputspectra/spec-00071.mgf,1033040.0,854.070,0,1074.53,5636,spec-00071.mgf,3,f.MSV000083508/ccms_peak/stomach/Trypsin_HCD_Q...,3.63020,3.63020,48,48,4001,MSV000083508/ccms_peak/stomach/Trypsin_HCD_QEx...,K.IREEYPDR.I,117109,26191.0,102,tr|Q5ST81|Q5ST81_HUMAN,0.000969
4,6,inputspectra/spec-00033.mgf,1434600.0,699.658,0,1074.69,3732,spec-00033.mgf,3,f.MSV000083508/ccms_peak/stomach/Trypsin_HCD_Q...,3.63020,3.63020,53,53,2101,MSV000083508/ccms_peak/stomach/Trypsin_HCD_QEx...,K.LKPGAPLRPK.L,149201,6604.0,101,tr|A0A087WWY3|A0A087WWY3_HUMAN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491914,1372561,inputspectra/spec-00058.mgf,85261.4,5351.320,0,2597.39,49866,spec-00058.mgf,2,f.MSV000083508/ccms_peak/stomach/Trypsin_HCD_Q...,1.14826,1.14826,38,38,43710,MSV000083508/ccms_peak/stomach/Trypsin_HCD_QEx...,"M.(A,41.013)TQVEPLLPGGATLLQAEEHGGLVR.K",21587,NaN,100,tr|A0A0B4J210|A0A0B4J210_HUMAN,0.001057
1491915,1372562,inputspectra/spec-00031.mgf,118631.0,6031.090,0,2597.41,54440,spec-00031.mgf,2,f.MSV000083508/ccms_peak/stomach/Trypsin_HCD_Q...,1.14826,1.14826,121,121,46866,MSV000083508/ccms_peak/stomach/Trypsin_HCD_QEx...,F.QIEETIDRETSGNLEQLLLAVVK.S,196055,23055.0,100,tr|D6RBE9|D6RBE9_HUMAN,0.001057
1491916,1372577,inputspectra/spec-00025.mgf,98456.4,6109.820,0,2598.15,52978,spec-00025.mgf,2,f.MSV000083508/ccms_peak/stomach/Trypsin_HCD_Q...,3.22637,3.22637,20,20,44643,MSV000083508/ccms_peak/stomach/Trypsin_HCD_QEx...,"R.(C,39.995)PETLFQPSFIGMESAGIHETTY.N",29253,15725.0,101,sp|P63267-2|ACTH_HUMAN,0.001057
1491917,1372577,inputspectra/spec-00054.mgf,-1.0,6089.000,0,2598.15,49850,spec-00054.mgf,2,f.MSV000083508/ccms_peak/stomach/Trypsin_HCD_Q...,3.22637,3.22637,35,35,40697,MSV000083508/ccms_peak/stomach/Trypsin_HCD_QEx...,"R.(C,39.995)PETLFQPSFIGMESAGIHETTY.N",29253,15725.0,101,sp|P63267-2|ACTH_HUMAN,0.001057


In [ ]:
df1.columns

Index(['cluster index', 'AllFiles', 'sum(precursor intensity)', 'RTMean',
       'RTStdErr', 'parent mass', 'ScanNumber', 'ProteosafeFilePath',
       'PrecursorCharge', 'Original_Path', 'Mean_KL', 'Max_KL',
       'Min_SNR2_Peaks', 'Max_SNR2_Peaks', 'Spectrum_index',
       'LorikeetFilePath', 'Peptide', 'Variant_group', 'Spectral_network',
       'Num_tags', 'Protein', 'FDR'],
      dtype='object')

In [ ]:
# creating a fixed length array by taking min and max index.
MIN_INDEX = 50
MAX_INDEX = 2050

# function first normalizes the values, then stores the values by checking the min and max index.
# the intensity array norm is transformed to represent the spectrum information in the range 50 to 2050 
# for simplicity the data is stored as a csr matrix.
def process_mzml(f, min_index = MIN_INDEX, max_index = MAX_INDEX, columns = ['scan_id','ms level','scanList','count']):
  def normalize(x):
    x = np.array(x)
    total = np.sum(x)
    return x/total
  def process(x):
    output = np.zeros(MAX_INDEX+1)
    for i,ele in enumerate(x['m/z array']):
        if int(ele) > MAX_INDEX:
            break
        output[int(ele)-MIN_INDEX] = max(output[int(ele)-MIN_INDEX],x['intensity array norm'][i])
    return output
  def sparsify(x):
    return csr_matrix(x)
  def get_scan(x):
    return int(re.search('scan=(\d+)',x['id']).group(0).replace('scan=',''))


  reader = mzml.read(f,read_schema=True)
  data = []
  try:
    while reader is not None:
      data.append(reader.next())
  except:
    print("Done reading mzml file. Processing file...")    
  df = pd.DataFrame(data)
  # normalize
  df['intensity array norm'] = df['intensity array'].apply(normalize)
  # convert to given range index (fixed length)
  df['spectrum'] = df.parallel_apply(process, axis = 1)
  # convert to a sparse matrix
  df['spectrum_sparse'] = df['spectrum'].apply(sparsify)
  # store scan id and spectrum info
  df['scan_id'] = df.apply(get_scan,axis=1)
  return df[columns], df[['spectrum_sparse','scan_id']]

# saving the spectrum data as a npz file
def save_spectrum(file, df):
  with open(file+'_spectrum.npz','wb') as f:
    np.savez(f,np.array(df))
# keeping the meta information of the spectrums like mass, charge, scanlist, scan id
def save_meta(file, df):
  df.to_csv(file+'_processed_meta.csv')

# merge the peptide information along with the spectrum meta information by joining on Scan ID.
def filter_and_merge(df, processed_meta, file):
  filtered = df[df['Original_Path']=='f.'+file]
  filtered['scan_id'] = filtered['ScanNumber']
  return df.merge(processed_meta, on ='scan_id')

# saving the final merged data file
def save_merged(file,df):
  df.to_csv(file+'_merged.csv')

In [ ]:
# connecting to ftp server to get mzml files
ftp = FTP('massive.ucsd.edu')  # connect to host, default port
ftp.login()                     # user anonymous, passwd anonymous@

'230 Login successful.'

In [ ]:
ftp.cwd('MSV000083508/ccms_peak/stomach/Trypsin_HCD_QExactiveplus')

'250 Directory successfully changed.'

In [ ]:
# getting the file names 
f = io.StringIO()
with redirect_stdout(f):
    ftp.dir()
s = f.getvalue()


In [ ]:
s = s.replace('\n',' ')
s = s.split()
files = []
for w in s:
  if '.mzML' in w:
    files.append(w)

files

['01087_A01_P010739_S00_N01_R1.mzML',
 '01087_A02_P010739_S00_N09_R1.mzML',
 '01087_A03_P010739_S00_N17_R1.mzML',
 '01087_A04_P010739_S00_N25_R1.mzML',
 '01087_A05_P010739_S00_N33_R2.mzML',
 '01087_B01_P010739_S00_N02_R1.mzML',
 '01087_B02_P010739_S00_N10_R1.mzML',
 '01087_B03_P010739_S00_N18_R1.mzML',
 '01087_B04_P010739_S00_N26_R1.mzML',
 '01087_B05_P010739_S00_N34_R2.mzML',
 '01087_C01_P010739_S00_N03_R1.mzML',
 '01087_C02_P010739_S00_N11_R1.mzML',
 '01087_C03_P010739_S00_N19_R1.mzML',
 '01087_C04_P010739_S00_N27_R1.mzML',
 '01087_C05_P010739_S00_N35_R2.mzML',
 '01087_D01_P010739_S00_N04_R1.mzML',
 '01087_D02_P010739_S00_N12_R1.mzML',
 '01087_D03_P010739_S00_N20_R1.mzML',
 '01087_D04_P010739_S00_N28_R1.mzML',
 '01087_D05_P010739_S00_N36_R1.mzML',
 '01087_E01_P010739_S00_N05_R1.mzML',
 '01087_E02_P010739_S00_N13_R1.mzML',
 '01087_E03_P010739_S00_N21_R1.mzML',
 '01087_E04_P010739_S00_N29_R1.mzML',
 '01087_F01_P010739_S00_N06_R1.mzML',
 '01087_F02_P010739_S00_N14_R1.mzML',
 '01087_F03_

In [ ]:
# taking 5 mzML files from the stomach tissue dataset
FILE_COUNT = 5
for f in files[:FILE_COUNT]:
  with open(f, 'wb') as fp:
    ftp.retrbinary('RETR '+f, fp.write)

In [ ]:
# performing the data processing steps on each file
# for each mzML file we get a spectrum.npz file, meta.csv, merged.csv
FILE_COUNT = 5
for f in files[:FILE_COUNT]:
  meta, sparse = process_mzml(f)
  save_spectrum(f,sparse)
  save_meta(f,meta)
  merged = filter_and_merge(df1,meta,f)
  save_merged(f, merged)

In [ ]:
# creating a definition to merge all the data_processed files into a single pickle
spectrum_extension = '.npz'
csv_extension = '_merged.csv'
csv_filenames = [i for i in glob.glob(f"*{csv_extension}")]
spectrum_filenames = [i for i in glob.glob(f"*{spectrum_extension}")]
filenames = zip(sorted(csv_filenames),sorted(spectrum_filenames))
data = []
for files in filenames:
  csv_file, spectrum_file = files
  print(csv_file)
  with open(spectrum_file,'rb') as f:
    spectrum = np.load(f, allow_pickle=True)
    spectrum = pd.DataFrame.from_dict({item: np.array(spectrum[item]) for item in spectrum.files},orient='index')
    spectrum = pd.DataFrame(spectrum[0]['arr_0'], columns=['spectrum','scan_id'])
  merged = pd.read_csv(csv_file)
  merged = merged[['scan_id','PrecursorCharge','parent mass','Peptide','Protein']].merge(spectrum,on='scan_id')
  data.append(merged)
# final pickle file containing the spectrum information, peptide, protein, scan id.
# this is used in the models to create the train, test and validation sets.
pd.concat(data).to_pickle('stomach_merged.pickle')